In [3]:
import requests
import pandas as pd
import xmltodict
import math

In [6]:
dim_doi = catalog.load('stg/dim_doi')
r_token = catalog.load('params:openaire_fetch_options.access_token')
env = 'dev'
pd.set_option("display.max_columns", None)


[01/28/25 14:47:45] INFO     Loading data from stg/dim_doi (SQLTableDataset)...                 ]8;id=482722;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=533212;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/core.py:2 │
│ 44 in load                                                                                       │
│                                                                                                  │
│   241 │   │   │   self._logger.debug("Loading %s", str(self))                                    │
│   242 │   │   │                                                                                  │
│   243 │   │   │   try:                                                                           │
│ ❱ 244 │   │   │   │   return load_func(self)                                                     │
│   245 │   │   │   except DatasetError:                                                           │
│   246 │   │   │   │   raise                                                                      │
│   247 │   │   │   except Exception as exc:                                                       │
│                                                                                                  │
│ /home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro_datasets/pan │
│ das/sql_dataset.py:266 in load                                                                   │
│                                                                                                  │
│   263 │   │   }                                                                                  │
│   264 │                                                                                          │
│   265 │   def load(self) -> pd.DataFrame:                                                        │
│ ❱ 266 │   │   return pd.read_sql_table(con=self.engine, **self._load_args)                       │
│   267 │                                                                                          │
│   268 │   def save(self, data: pd.DataFrame) -> None:                                            │
│   269 │   │   data.to_sql(con=self.engine, **self._save_args)                                    │
│                                                                                                  │
│ /home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/pandas/io/sql.py:3 │
│ 86 in read_sql_table                                                                             │
│                                                                                                  │
│    383 │                                                                                         │
│    384 │   with pandasSQL_builder(con, schema=schema, need_transaction=True) as pandas_sql:      │
│    385 │   │   if not pandas_sql.has_table(table_name):                                          │
│ ❱  386 │   │   │   raise ValueError(f"Table {table_name} not found")                             │
│    387 │   │                                                                                     │
│    388 │   │   table = pandas_sql.read_table(                                                    │
│    389 │   │   │   table_name,                                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Table dim_doi not found

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 dim_doi = catalog.load('stg/dim_doi')                                                        │
│   2 r_token = catalog.load('params:openaire_fetch_opt

In [5]:
dim_doi

'REPLACE_ME'

### Recupero datos de OpenAire Graph

Hago una pequeña manipulación de datos, que debería evitarse. Para eso en vez de el hub doi, hay que usar una dim_doi con el valor del pid ya manipulado en mart.



In [4]:
def fetch_researchproduct_openaire(dim_doi: pd.DataFrame, r_token, env)-> pd.DataFrame:
    base_url = "https://api.openaire.eu/search/researchProducts"
    df_list = []

    doi_limit = 9999
    if (env == 'dev'): doi_limit = 9
    
    skipped_list = []

    not_in_openaire = dim_doi['in_openaire'] == False
    dim_doi = dim_doi[not_in_openaire]

    doi_list = dim_doi.iloc[0:doi_limit]['doi'].to_list()
    doi_comma_separated = ','.join(doi_list)

    # se define cantidad de batches a partir de la cantidad de resultados por batch y cantidad de doi
    batch_size = 10
    num_batches = math.ceil(len(doi_list) / batch_size)

    for batch_index in range(num_batches):

        batch = doi_list[batch_index * batch_size : (batch_index + 1) * batch_size]
        doi_comma_separated = ','.join(batch)

        graph_url = f"{base_url}?doi={doi_comma_separated}"
        headers = { 'Authorization': f'Bearer {r_token}' }

        api_response = requests.get(graph_url, headers=headers)
        print(f'GET "{graph_url}" {api_response.status_code}')

        if api_response.status_code == 200:
            data_dict = xmltodict.parse(api_response.content)
            results = data_dict.get('response', {}).get('results', {}).get('result', [])

            for result in results:

                publication_header = result.get('header', {})
                publication_metadata = result.get('metadata', {}).get('oaf:entity', {}).get('oaf:result', {})

                publication = publication_header | publication_metadata 
                if publication:
                    df_normalized = pd.json_normalize(publication, max_level=0)
                    df_list.append(df_normalized)
                else:
                    print("No publication data found in result")
        else:
            print(f'Error: Received status code {api_response.status_code}')
            skipped_list.extend(batch)
            break

    print(f'{len(df_list)} batches processed')
    print(f'{len(skipped_list)} DOIs skipped')

    if df_list:
        df = pd.concat(df_list, ignore_index=True)
    else:
        df = pd.DataFrame()   

    return df, skipped_list


In [5]:
df, skiped_list = fetch_researchproduct_openaire(dim_doi, r_token, env)


GET "https://api.openaire.eu/search/researchProducts?doi=10.24267/22564004.544,10.5151/despro-sigradi2016-510,10.1051/0004-6361/201732349,10.1016/j.revpalbo.2021.104401,10.4322/polimeros.2013.033,10.3390/quantum4010009,10.1111/j.1472-765x.2005.01785.x,10.1016/j.foodqual.2007.06.004,10.1016/j.asr.2020.07.008" 200
9 batches processed
0 DOIs skipped


In [6]:
df

,@xmlns:xsi,dri:objIdentifier,dri:dateOfCollection,dri:dateOfTransformation,collectedfrom,originalId,pid,measure,fulltext,title,bestaccessright,creator,country,dateofacceptance,description,subject,language,relevantdate,publisher,source,format,resulttype,resourcetype,isgreen,openaccesscolor,isindiamondjournal,publiclyfunded,journal,datainfo,rels,children,context,contributor
0,http://www.w3.org/2001/XMLSchema-instance,doi_dedup___::2577d684a839dc78be3e1307914cdfd5,2024-10-11T14:10:24.466,None,[{'@name': 'LAReferencia - Red Federada de Rep...,"[quantum4010009, 10.3390/quantum4010009, 50|do...","{'@classid': 'doi', '@classname': 'Digital Obj...","[{'@id': 'influence', '@score': '3.29944E-9', ...",http://www.mdpi.com/2624-960X/4/1/9/pdf,"[{'@classid': 'main title', '@classname': 'mai...","{'@classid': 'OPEN', '@classname': 'Open Acces...","[{'@rank': '1', '@name': 'Diana', '@surname': ...","{'@classid': 'AR', '@classname': 'Argentina', ...",2022-02-22,<jats:p>We deal here with an exactly solvable ...,"[{'@classid': 'keyword', '@classname': 'keywor...","{'@classid': 'und', '@classname': 'Undetermine...","[{'@classid': 'created', '@classname': 'create...",MDPI AG,"[Crossref, Quantum Reports; Volume 4; Issue 1;...","[application/pdf, 127-134]","{'@classid': 'publication', '@classname': 'pub...","{'@classid': 'publication', '@classname': 'pub...",true,gold,false,false,"{'@eissn': '2624-960X', '@ep': '134', '@sp': '...","{'inferred': 'true', 'deletedbyinference': 'fa...","{'rel': [{'@inferred': 'true', '@trust': '0.65...",{'result': [{'@objidentifier': 'doajarticles::...,NaN,NaN
1,http://www.w3.org/2001/XMLSchema-instance,doi_dedup___::fccba469c4dbc14c5cec34e69ab21625,2024-10-11T14:20:20.957,None,"[{'@name': 'Crossref', '@id': 'openaire____::0...","[S0273117720304853, 10.1016/j.asr.2020.07.008,...","[{'@classid': 'doi', '@classname': 'Digital Ob...","[{'@id': 'influence', '@score': '2.95062E-9', ...",NaN,"[{'@classid': 'main title', '@classname': 'mai...","{'@classid': 'CLOSED', '@classname': 'Closed A...","[{'@rank': '1', '#text': 'Nancy Esther Quarant...",NaN,2020-10-01,"Abstract In a first paper, we presented an i...","[{'@classid': 'FOS', '@classname': 'Fields of ...","{'@classid': 'und', '@classname': 'Undetermine...","[{'@classid': 'created', '@classname': 'create...",Elsevier BV,Crossref,NaN,"{'@classid': 'publication', '@classname': 'pub...","{'@classid': 'publication', '@classname': 'pub...",false,bronze,false,false,"{'@issn': '0273-1177', '@ep': '2000', '@sp': '...","{'inferred': 'true', 'deletedbyinference': 'fa...","{'rel': [{'@inferred': 'false', '@trust': '0.9...",{'result': {'@objidentifier': 'doi_________::f...,"{'@id': 'energy-planning', '@label': 'Energy P...",NaN
2,http://www.w3.org/2001/XMLSchema-instance,doi_dedup___::000d1effb7dd214508cfa2db791e3814,2024-10-11T14:28:54.171,None,[{'@name': 'LAReferencia - Red Federada de Rep...,"[10.24267/22564004.544, 50|doiboost____|000d1e...","[{'@classid': 'doi', '@classname': 'Digital Ob...","[{'@id': 'influence', '@score': '2.8970693E-9'...",NaN,"[{'@classid': 'main title', '@classname': 'mai...","{'@classid': 'OPEN', '@classname': 'Open Acces...","[{'@rank': '1', '@name': 'Federico', '@surname...","{'@classid': 'AR', '@classname': 'Argentina', ...",2021-07-09,<jats:p>El objetivo de este artículo es analiz...,"[{'@classid': 'keyword', '@classname': 'keywor...","{'@classid': 'und', '@classname': 'Undetermine...","[{'@classid': 'created', '@classname': 'create...",Universidad de Boyacá,"[Crossref, reponame:SEDICI (UNLP), instname:Un...","[application/pdf, 27-47]","{'@classid': 'publication', '@classname': 'pub...","{'@classid': 'publication', '@classname': 'pub...",true,gold,false,false,"{'@issn': '2256-4004', '@eissn': '2539-147X', ...","{'inferred': 'true', 'deletedbyinference': 'fa...","{'rel': {'@inferred': 'false', '@trust': '0.9'...",{'result': [{'@objidentifier': 'doi_________::...,"{'@id': 'energy-planning', '@label': 'Energy P...",NaN
3,http://www.w3.org/2001/XMLSchema-